# KML do nalotów

widgety do wprowadzania danych

In [1]:
#setup UI
import ipywidgets as widgets
lat = 50.936667
wLat = widgets.BoundedFloatText(
 value=lat,
 min=0,
 max=60,
 description='Lat:'
 )
lon = 16.861389
wLon = widgets.BoundedFloatText(
 value=16.861389,
 min=0,
 max=20,
 description='Lon:'
 )
radius = 50
wRadius = widgets.BoundedFloatText(
 value=radius,
 min=0,
 max=200,
 description='Radius:'
 )
wNumpoints = widgets.BoundedIntText(
 value=16,
 min=2,
 max=100,
 description='Num Points:'
 )
wNumaxes = widgets.BoundedIntText(
 value=8,
 min=2,
 max=20,
 description='Num Axes:'
 )
bCalculate = widgets.Button(
    description='Policz',
    icon='check' # (FontAwesome names without the `fa-` prefix),
)

In [2]:
#setup map
from ipyleaflet import Map, basemaps, basemap_to_tiles,LayersControl,ScaleControl,MeasureControl,\
  LayerGroup,Marker, Polygon 


mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
mapnik.base = True
mapnik.name = 'Mapnik Layer'

toner = basemap_to_tiles(basemaps.Stamen.Toner)
toner.base = True
toner.name = 'Toner Layer'

bzh = basemap_to_tiles(basemaps.OpenStreetMap.BZH)
bzh.base = True
bzh.name = 'BZH layer'

mapa =Map(center = (wLat.value, wLon.value), zoom = 17, min_zoom = 1, max_zoom = 20, 
    layers=[mapnik, toner, bzh])

measure = MeasureControl(
    position='bottomright',
    active_color = 'orange',
    primary_length_unit = 'meters',
    primary_area_unit = 'sqmeters',
    completed_color = 'red'
)
mapa.add_control(measure)
mapa.add_control(ScaleControl(position='bottomleft'))
mapa.add_control(LayersControl(position='topright'))

centerMarker = Marker(location=(wLat.value, wLon.value), draggable=True, name="Środek")
def centerMarker_on_location_changed(event):
  wLat.value, wLon.value = centerMarker.location

centerMarker.observe(centerMarker_on_location_changed, 'location')
mapa.add_layer(centerMarker)
pointsLayer = LayerGroup(name = 'Linie')
mapa.add_layer(pointsLayer)


In [3]:
#setup calculations
import math
from pyproj import Transformer, CRS, transform
points =[]
def calculate():
  global points
  global lat, lon, radius
  num_points = wNumpoints.value
  num_axes = wNumaxes.value
  lat = wLat.value
  lon = wLon.value
  radius = wRadius.value


  #Source projection
  sourceProj = CRS('epsg:4326')
  #Azimuthal Equidistant projection centred on your origin point
  localProj = CRS(proj="aeqd", lat_0=lat, lon_0=lon, datum="WGS84", units="m")
  transformer = Transformer.from_crs(sourceProj,localProj)
  invtransformer = Transformer.from_crs(localProj,sourceProj)
  
  x,y = transformer.transform(lat, lon)
  angle = 2 * math.pi / num_points

  pointsLocal = [( 
    x + (radius * math.cos(i * angle)),
    y + (radius * math.sin(i * angle))
    ) for i in range(num_points)]
  points = [*invtransformer.itransform(pointsLocal)]

  #angleAxes = 2 * math.pi / num_axes
  # pointMap = map(lambda i: [ 
  #         lat + (radius * math.cos(i * angle)),
  #         lon + (radius * math.sin(i * angle))]
  #         , range(wNumpoints.value))
  # points = list(pointMap)
  centerMarker.location =(lat,lon)
  mapa.center = (lat,lon)
  linia = Polygon(locations=points,
    color="green" ,
    fill=True)
  pointsLayer.clear_layers()
  pointsLayer.add_layer(linia) 
 
def bCalculate_on_click(e):
  calculate()
bCalculate.on_click(bCalculate_on_click)
  

In [4]:
#UI
display(wLat,wLon,wRadius,wNumpoints, wNumaxes,bCalculate)
display(mapa)

BoundedFloatText(value=50.936667, description='Lat:', max=60.0)

BoundedFloatText(value=16.861389, description='Lon:', max=20.0)

BoundedFloatText(value=50.0, description='Radius:', max=200.0)

BoundedIntText(value=16, description='Num Points:', min=2)

BoundedIntText(value=8, description='Num Axes:', max=20, min=2)

Button(description='Policz', icon='check', style=ButtonStyle())

Map(center=[50.936667, 16.861389], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [5]:
#simple kml
import simplekml
def getkml():
  kml = simplekml.Kml(name = "Nalot")
  for idx, p in enumerate(points):
    #kml lon, lat , alt - lon WE, lat -NS czyli inaczej niż wyżej
    kml.newpoint(name=idx+1, coords=[[*reversed(p)]])
  return kml


In [6]:
#pykml
from pykml.factory import nsmap
from pykml.factory import KML_ElementMaker as KML
from pykml.factory import GX_ElementMaker as GX
from pykml.parser import Schema
from lxml import etree

def prinstSaveKml2():
  # define a variable for the Google Extensions namespace URL string
  gxns = '{' + nsmap['gx'] + '}'

  feature_list = [
    {
        'name':"Punkt",
        'desc':'Wokół tego ma latać',
        'lon': lon,'lat': lat, 'tilt':45, 'range':radius,
    } #,
    # {
    #     'name':"Ayers Rock",
    #     'desc':'Northern Territory, central Australia',
    #     'lon':131.035,'lat':-25.345, 'tilt':88, 'range':5000,
    # },
    # {
    #     'name':"Landscape Arch",
    #     'desc':'Arches National Park, Utah, USA',
    #     'lon':-109.607,'lat':38.791, 'tilt':60, 'range':500,
    # },
    # {
    #     'name':"Manicouagan Crater",
    #     'desc':'Quebec, Canada',
    #     'lon':-68.7, 'lat':51.38, 'tilt':45, 'range':100000,
    # },
    # {
    #     'name':"Upheaval Dome",
    #     'desc':'Canyonlands National Park, Utah, USA',
    #     'lon':-109.929, 'lat':38.437, 'tilt':60, 'range':4000,
    # },
    # {
    #     'name':"Racetrack Playa",
    #     'desc':'Death Valley, California, USA',
    #     'lon':-117.56, 'lat':36.679, 'tilt':45, 'range':8000
    # },
    # {
    #     'name':"Matterhorn",
    #     'desc':'Pennine Alps, Switzerland/Italy',
    #     'lon':7.6584, 'lat':45.9766, 'tilt':60, 'range':6000,
    # },
  ]
  # start with a base KML tour and playlist
  tour_doc = KML.kml(
    KML.Document(
      GX.Tour(
        KML.name("Play me!"),
        GX.Playlist(),
      ),
      KML.Folder(
        KML.name('Features'),
        id='features',
      ),
    )
  )
  for feature in feature_list:
    #import ipdb; ipdb.set_trace()
    # fly to a space viewpoint
    tour_doc.Document[gxns+"Tour"].Playlist.append(
      GX.FlyTo(
        GX.duration(5),
        GX.flyToMode("smooth"),
        KML.LookAt(
          KML.longitude(feature['lon']),
          KML.latitude(feature['lat']),
          KML.altitude(0),
          KML.heading(0),
          KML.tilt(0),
          KML.range(10000000.0),
          KML.altitudeMode("relativeToGround"),
        )
      ),
    )
    # fly to the feature
    tour_doc.Document[gxns+"Tour"].Playlist.append(
      GX.FlyTo(
        GX.duration(5),
        GX.flyToMode("bounce"),
        KML.LookAt(
          KML.longitude(feature['lon']),
          KML.latitude(feature['lat']),
          KML.altitude(0),
          KML.heading(0),
          KML.tilt(feature['tilt']),
          KML.range(feature['range']),
          KML.altitudeMode("relativeToGround"),
        )
      ),
    )
    # spin around the feature
    for aspect in range(0,360,10):
        tour_doc.Document[gxns+"Tour"].Playlist.append(
          GX.FlyTo(
            GX.duration(0.25),
            GX.flyToMode("smooth"),
            KML.LookAt(
              KML.longitude(feature['lon']),
              KML.latitude(feature['lat']),
              KML.altitude(0),
              KML.heading(aspect),
              KML.tilt(feature['tilt']),
              KML.range(feature['range']),
              KML.altitudeMode("relativeToGround"),
            )
          )
        )
    tour_doc.Document[gxns+"Tour"].Playlist.append(GX.Wait(GX.duration(1.0)))
    
  #    tour_doc.Document[gxns+"Tour"].Playlist.append(
  #        GX.TourControl(GX.playMode("pause"))
  #    )
    
    # add a placemark for the feature
    tour_doc.Document.Folder.append(
      KML.Placemark(
        KML.name("?"),
        KML.description(
            "<h1>{name}</h1><br/>{desc}".format(
                    name=feature['name'],
                    desc=feature['desc'],
            )
        ),
        KML.Point(
          KML.extrude(1),
          KML.altitudeMode("relativeToGround"),
          KML.coordinates("{lon},{lat},{alt}".format(
                  lon=feature['lon'],
                  lat=feature['lat'],
                  alt=50,
              )
          )
        ),
        id=feature['name'].replace(' ','_')
      )
    )
    # show the placemark balloon
    tour_doc.Document[gxns+"Tour"].Playlist.append(
        GX.AnimatedUpdate(
          GX.duration(2.0),
          KML.Update(
            KML.targetHref(),
            KML.Change(
              KML.Placemark(
                KML.visibility(1),
                GX.balloonVisibility(1),
                targetId=feature['name'].replace(' ','_')
              )
            )
          )
        )
    )
    
    tour_doc.Document[gxns+"Tour"].Playlist.append(GX.Wait(GX.duration(2.0)))
    
    tour_doc.Document[gxns+"Tour"].Playlist.append(
        GX.AnimatedUpdate(
          GX.duration(2.0),
          KML.Update(
            KML.targetHref(),
            KML.Change(
              KML.Placemark(
                GX.balloonVisibility(0),
                targetId=feature['name'].replace(' ','_')
              )
            )
          )
        )
    )
    # fly to a space viewpoint
    tour_doc.Document[gxns+"Tour"].Playlist.append(
      GX.FlyTo(
        GX.duration(5),
        GX.flyToMode("bounce"),
        KML.LookAt(
          KML.longitude(feature['lon']),
          KML.latitude(feature['lat']),
          KML.altitude(0),
          KML.heading(0),
          KML.tilt(0),
          KML.range(10000000.0),
          KML.altitudeMode("relativeToGround"),
        )
      ),
    )

  # check that the KML document is valid using the Google Extension XML Schema
  #assert(Schema("kml22gx.xsd").validate(tour_doc))

  kml_xml = etree.tostring(tour_doc, pretty_print=True, encoding = "unicode")
  print(kml_xml)

  # output a KML file (named based on the Python script)
  with open("punkty2.kml", "w") as kml_file:
    kml_file.write(kml_xml)

In [8]:
#kml ui
bGenerateKml = widgets.Button(
    description='Pokaż KML',
    icon='cloud' # (FontAwesome names without the `fa-` prefix),
)
bSaveKml = widgets.Button(
    description='Zapisz KML',
    icon='floppy-o',
)
bKml2 = widgets.Button(
    description='Inny KML',
    icon='ravelry',
)

def bGenerateKml_on_click(e):
  print(getkml().kml())
bGenerateKml.on_click(bGenerateKml_on_click)

def bSaveKml_on_click(e):
  getkml().save("punkty.kml")
bSaveKml.on_click(bSaveKml_on_click)

def bKml2_on_click(e):
  prinstSaveKml2()
bKml2.on_click(bKml2_on_click)

prinstSaveKml2
display(bGenerateKml,bSaveKml,bKml2)


Button(description='Pokaż KML', icon='cloud', style=ButtonStyle())

Button(description='Zapisz KML', icon='floppy-o', style=ButtonStyle())

Button(description='Inny KML', icon='ravelry', style=ButtonStyle())